# 03 Build Macro Quarterly Features

Aggregate monthly predictors to quarterly, add lags, and merge with targets.


## Table of Contents
- Aggregate monthly -> quarterly
- Add lags
- Merge with GDP/labels
- Save macro_quarterly.csv


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## What You Will Produce
- data/processed/macro_quarterly.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Matching Guide
- `docs/guides/01_data/03_build_macro_quarterly_features.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/01_data/03_build_macro_quarterly_features.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Aggregate monthly predictors to quarterly features (mean vs last), add lags, and merge with GDP + targets.


## Your Turn: Load panel_monthly + gdp_quarterly


In [ ]:
from src import data as data_utils

# TODO: Load panel_monthly.csv and gdp_quarterly.csv
...


## Your Turn: Quarterly Aggregation


In [ ]:
from src import macro

# TODO: Create quarterly features using BOTH methods: mean and last
# Compare them visually/statistically
...


## Your Turn: Add Lags


In [ ]:
# TODO: Add quarterly lags (1, 2, 4) for each predictor
...


## Your Turn: Merge and Save


In [ ]:
# TODO: Merge predictors with GDP growth + recession labels
# Save to data/processed/macro_quarterly.csv
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/macro_quarterly.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Aggregate monthly -> quarterly</summary>

```python
import pandas as pd
from src import macro

panel_monthly = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)
q_mean = macro.monthly_to_quarterly(panel_monthly, how='mean')
q_last = macro.monthly_to_quarterly(panel_monthly, how='last')
q_mean.head(), q_last.head()
```

</details>

<details><summary>Solution: Add lags</summary>

```python
q = q_mean.copy()
for col in q.columns:
    for lag in [1, 2, 4]:
        q[f'{col}_lag{lag}'] = q[col].shift(lag)
q = q.dropna()
```

</details>

<details><summary>Solution: Merge with GDP/labels</summary>

```python
gdp = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)
df = q.join(gdp, how='inner').dropna()
df.head()
```

</details>

<details><summary>Solution: Save macro_quarterly.csv</summary>

```python
from src import data as data_utils
data_utils.save_csv(df, PROCESSED_DIR / 'macro_quarterly.csv')
print('saved', PROCESSED_DIR / 'macro_quarterly.csv')
```

</details>

